In [1]:
from N3_FL import *
from json import loads
from U2_Merkle import *

In [2]:
def truncateRepus(repus):
    nrepus = []
    for r in repus:
        nrepus.append({'hash': r['hash'][:5], 'qual_mined' : "{:.2f}".format(r['qual_mined']), 'qual_reported' : "{:.2f}".format(r['qual_reported'])})
    return nrepus

In [3]:
# Add bogus payment model
class Transaction:
    def __init__ (self, chain, sender, recipient, load, type_, payment): #type_ - mod/pay/glb
        self.sender = sender
        self.load = load
        self.recipient = recipient
        self.type = type_
        self.payment = payment
        self.chain = chain
    
    def __str__(self):
        r = "TRX: C{0:5.5}:{1:5.5}-->{2:5.5} | {3:5.5} | ¤{4:4.2f} | Load: ".format(str(self.chain), str(self.sender), str(self.recipient), self.type, self.payment) + self.load
        return r

In [4]:
class Block:
    def __init__(self, chain, chain_idx, trxs, proof, prev_hash, ts=None, repu_list=None):
        self.chain = chain
        self.chain_idx = chain_idx
        self.transactions = trxs
        self.proof = proof
        self.prev_hash = prev_hash
        if ts == None:
            self.timestamp = time()
        else:
            self.timestamp = ts
        if repu_list == None:
            self.reputation_list = []
        else:
            self.reputation_list = repu_list
        print(trxs)
        self.merkle = buildTree([toHash(x) for x in trxs])
        self.tree_hash = objPickleToIPFS(IPFS(0), self.merkle, "./temp")

    def __str__(self):
        r = "BLK: C:{0:7.7}#{1:3d} <-- PH={2:5.5}:PF={3:5.5}, RL={4:5.5}, #TRXS={5}".format(str(self.chain.bc_id),self.chain_idx,str(self.prev_hash),str(self.proof),str(self.reputation_list), str(len(self.transactions)))
        return r

In [5]:
def removeCli(txns, cli): #in dict
    tx_new = []
    for t in txns:
        if t.type == 'PAY':
            tx_new.append(t)
        if t.type == 'MOD':
            if t.sender == cli:
                pass
            else:
                tx_new.append(t)
    return tx_new

def truncateViewRL(rl):
    r = ""
    for c, q in rl.items():
        r += "{0:5.5}:{1:.4}; ".format(c,q)
    return r

In [6]:
# MAKE CONVERTER TO OTHER ROLE VIA TRX DATA?????
class Node(Client):
    def __init__(self, id_):
        super().__init__(id_)
        self.blockchain = None
        self.balance = 0
        self.committee = True
        self.mine_benchmarks = []
        self.current_trxs = []
        self.new_repus = []

    def initSet(self):
        if hasattr(self, 'blockchain') == False:
            self.blockchain = None
            self.balance = 0
    
    def __str__(self):
        return "{0}:{3}-{1:5.5}, {2:7d}".format(self.getType(), \
                            self.hash, self.getVolume(), self.id_)

    def getType(self):
        return "NDE"

    def assignBlockchain(self, blockchain):
        self.blockchain = blockchain
        self.blockchain.authorizeClient(self)
                        
    def volAsHash(self):
        loc_vol = './temp/t1.txt'
        vol_hash = self.ipfs.sendToIPFS(saveToFile(str(self.getVolume()/self.roundLimit), loc_vol))
        rm(loc_vol)
        return vol_hash

    def submitModelAsTransaction(self):
        # send model HASH and MULTIPLIER (number of data)
        load = "{}|{}|{}|{}".format(self.hash, self.hashes[0], self.hashes[1], self.volAsHash()) # mod, qual, vol
        trx = Transaction(self.blockchain.bc_id, self.hash, 0, load, 'MOD', 0)
        chidx = self.blockchain.newTransaction(trx)
        print("{0}:{1} MOD-->CHIDX#{2:3d}".format(self.getType(),str(self.id_), chidx))

    def evaluateMine(self, model_weights):
        model_ = CNN()
        model_.setup()
        model_.replaceModel(model_weights)
        qual = model_.model.evaluate(self.test_inp, self.test_oup, verbose=2)
        return qual[1]
    
    def cliTrainPack(self, epoch_, r):
        loc = './models/'
        self.train(loc, epoch_, r)
        self.submitModelAsTransaction()

# COMMITTEE FUNCTIONS_________________________________________________________________________________
# WRONG AGGREGATION
    def aggregate(self, blockchain):
        if self.committee == False:
            return
    
        # Initiate resulting global model
        global_model = None
        
        loc = './models/'
        self.mine_benchmarks = []
        # IMPORT PAST QUALITIES
        # REPU LIST: QUAL (VOL * ACC)
        past_qualities = blockchain.lastBlock.reputation_list

        for t in blockchain.current_transactions:
            if t.type == 'MOD':
                # parse t
                cli_hash, mod_hash, qual_hash, vol_hash = t.load.split('|')
                model_ = IPFStoModObj(self.ipfs, mod_hash, loc)
                vol_ = int(float(IPFStoObj(self.ipfs, vol_hash, loc)))
                qual_ = float(IPFStoObj(self.ipfs, qual_hash, loc))
                
                # Test everything against itself
                # qual_ = REPORTED qual = vol ** accu
                # compare to c.evalmine
                benchmarks = {'hash': cli_hash, 'qual_mined' : self.evaluateMine(model_) * (vol_ ** 0.5), \
                                'vol' : vol_, 'qual_reported' : qual_, 'model_file' : model_.get_weights()}
                self.mine_benchmarks.append(benchmarks)

        # Quality check from previous versions of qualities
        # calculate past qualities average first

        # get sum of past qualities
        sum_qual = 0
        for record in past_qualities:
            sum_qual += record['qual_mined']

        past_average_qual = 0
        for record in past_qualities:
            past_average_qual += record['qual_mined'] / sum_qual

        improvement_threshold = 0.9
        average_threshold = 0.3
        bm_pass = []
        for bm in self.mine_benchmarks:
            if len(past_qualities) > 0:
                for record in past_qualities: # THIS ONE WONT RUN IF EMPTY
                    if bm['hash'] in record['hash']:
                        # LOC_NOW vs LOC_PAST
                        if float(bm['qual_mined']) > improvement_threshold * float(record['qual_mined']):
                            bm_pass.append(bm)
                            continue
                        # LOC_NOW vs GLOB_PAST
                        if float(bm['qual_mined']) > average_threshold * past_average_qual:
                            bm_pass.append(bm)
                    else:
                        bm_pass.append(bm)
            else:
                bm_pass.append(bm)
        self.mine_benchmarks = bm_pass
        print('MINE BENCHMARKS: ', truncateRepus(self.mine_benchmarks))

        sel_hashes = []
        for h in self.mine_benchmarks:
            sel_hashes.append(h['hash'])

        # Aggregate from benchmarks
        # formula: qual_report + qual_mined
        if len(self.mine_benchmarks) > 1:
            models_multiply = []
            models_multiply_proportion = []
            models = []    
            for bm in self.mine_benchmarks:
                # get weights and models
                models_multiply.append(bm['qual_mined'] + bm['qual_reported'])
                models.append(bm['model_file'])
            
            for m in models_multiply:
                models_multiply_proportion.append(m / sum(models_multiply))
            
            average = combine_ultimate(models, models_multiply_proportion)

            # aggregate
            global_model = CNN()
            global_model.setup()
            global_model.model.set_weights(average)
            global_model.setData(self.train_inp, self.train_oup)
            global_model.train('', getSession())
        elif len(self.mine_benchmarks) == 1:
            global_model = CNN()
            global_model.setup()
            global_model.model.set_weights(self.mine_benchmarks[0]['model_file'])
            global_model.setData(self.train_inp, self.train_oup)
            global_model.train('', getSession())
        else:
            #NO MINE
            print('Nothing to mine')

        print('GLOB EVAL ', self.evaluateMine(global_model.model))

        # REMOVE TRX FROM THOSE NOT IN FINAL C MINE BENCHMARKS (FILTER)
        trx_new = []
        for t in self.blockchain.current_transactions:
            if t.sender in sel_hashes or t.type == 'GLO':
                trx_new.append(t)

        hash_weight_global = objPickleToIPFS(self.ipfs, global_model.model.get_weights(), './temp/')
        mine_benchmarks = objPickleToIPFS(self.ipfs, truncateRepus(self.mine_benchmarks), './temp/')
        
        trx_global = Transaction(self.blockchain, self.hash, 0, "{}|{}|{}".format(self.hash, hash_weight_global, mine_benchmarks), 'GLO', 0)
        
        self.current_trxs = trx_new      
        self.current_trxs.append(trx_global)
        self.blockchain.newTransaction(trx_global)


In [7]:
class Compounder(Node):
    def __init__(self, id_):
        super().__init__(id_)
        self.ocNodes = []
        self.model_eval = CNN()
        self.model_eval.setup()
    
    def initSet(self):
        if hasattr(self, 'ocNodes') == False:
            self.ocNodes = []
            self.model_eval = CNN()
            self.model_eval.setup()

    def getType(self):
        return "CPD"

    def registerOCNode(self, node):
        self.ocNodes.append(node)

    def examine(self):
        m_evals = []
        m_lens = []
        m_mods = []
        for oc in self.ocNodes:
            self.model_eval.replaceModel(oc.model.model)
            mo = oc.model.model.get_weights()
            le = oc.getVolume()
            ev = self.model_eval.model.evaluate(self.test_inp, self.test_oup, verbose=2)
            m_evals.append(ev)
            m_lens.append(le)
            m_mods.append(mo)
        m_evals.append(self.model.model.evaluate(self.test_inp, self.test_oup, verbose=2))
        m_lens.append(self.getVolume())
        m_mods.append(self.model.model.get_weights())
        return m_mods, m_lens, m_evals

    def train(self, loc, epoch_, round_):
        self.model.model.set_weights(self.combineModels())
        return super().train(loc, epoch_, round_)

    # NEED TO HAVE SOME OFFCHAIN MODIF
    def replaceWeights(self, weights):
        self.model.model.set_weights(weights)
        #self.model.model.build((None,self,36,1))
        self.model.model.build((None,self,28,28,1))
        self.model.model.compile(optimizer='adam',loss='mse', metrics=['accuracy'])
        for oc in self.ocNodes:
            oc.replaceWeights(weights)

    def cliTrainPack(self, epoch_, r):
        loc = './models/'
        for oc in self.ocNodes:
            oc.train(loc, epoch_, r)
        self.train(loc, epoch_, r)
        self.submitModelAsTransaction()

    def combineModels(self):
        c_mods, c_lens, c_evals = self.examine()
        c_contribs = []
        c_props = []
        
        for ci in range(len(c_evals)):
            c_contribs.append(c_evals[ci][1] * (c_lens[ci] ** 0.5))

        for ci in range(len(c_evals)):
            c_props.append(c_contribs[ci]/sum(c_contribs))
        
        mod_combined = combine_ultimate(c_mods, c_props)
        return mod_combined

In [8]:
class OffChainNode(Client):
    def __init__(self, id_):
        super().__init__(id_)
        self.id_ = toHash(id_)

    def initSet(self):
        pass
    
    def getType(self):
        return "OCN"

    def __str__(self):
        return "{0}:{3}-{1:5.5}, {2:7d}".format(self.getType(), \
                            self.hash, self.getVolume(), self.id_)
    
    def requestBind(self, cli):
        cli.registerOCNode(self)

In [9]:
def setupClisF(train_inps_, train_oups_, test_inps_, test_oups_, rounds_, client_poi, classes_initial):
    clients = []
    for i in range(len(train_inps_)):
        c = classes_initial[i](i)
        c.model.setup()
        c.setData(train_inps_[i], train_oups_[i], test_inps_[i], test_oups_[i])
        c.setRoundLimit(rounds_)
        if i in client_poi:
            c = poisonFL(c)
        clients.append(c)
    return clients
    
def bindOCNodes(binds, clients):
    # binds: [cliOC, cliComp]
    for b in binds:
        clients[b[0]].requestBind(clients[b[1]])

In [10]:
model_dir = "./models/"
count = 7
test_ratio = 0.25
epochs = 2
rounds = 2
client_poisoning = [[], [5,4],[6,4,3],[1],[4,2,1],[0],[2,0]]
ep_it = [(2,1),(4,1),(8,1),(16,1),(1,2),(1,4),(2,2),(4,2),(2,4),(4,4),(8,2),(2,8)]

try:
    train_inps = pickle.load(open("./temp/trinps", "rb"))
    train_oups = pickle.load(open("./temp/troups", "rb"))
    test_inps = pickle.load(open("./temp/teinps", "rb"))
    test_oups = pickle.load(open("./temp/teoups", "rb"))
    test_inp = pickle.load(open("./temp/teinp", "rb"))
    test_oup = pickle.load(open("./temp/teoup", "rb"))
except:
    train_inp, test_inp, train_oup, test_oup = split(fetch(), test_ratio)
    train_inps = buildPortions(count, train_inp)
    train_oups = buildPortions(count, train_oup)
    test_inps = buildPortions(count, test_inp)
    test_oups = buildPortions(count, test_oup)
    pickle.dump(train_inps, open("./temp/trinps", "wb"))
    pickle.dump(train_oups, open("./temp/troups", "wb"))
    pickle.dump(test_inps, open("./temp/teinps", "wb"))
    pickle.dump(test_oups, open("./temp/teoups", "wb"))
    pickle.dump(test_inp, open("./temp/teinp", "wb"))
    pickle.dump(test_oup, open("./temp/teoup", "wb"))

In [11]:
def saveModelH(model, ipfs):
    save_loc = './models/glob0'
    model.model.save(save_loc, save_format='h5')
    hash = ipfs.sendToIPFS(save_loc)
    rm(save_loc)
    return hash

def saveFileToLoc(content, loc):
    f = open(loc, 'w')
    f.write(content)
    f.close()
    return loc

In [12]:
class Blockchain(object): # PROVING NODE: ID + SALT + HASH
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        self.bc_id = toHash(id(self))
        self.clients = set()
        self.cli_pointers = []
        self.newBlock(Block(self,0, [Transaction(self, 0, 0, "Genesis", "GEN", 0)], 0, 0, []))
        self.salt = str(randint(1,100000)) # salt for this blockchain
        print("Blockchain created with ID: " + self.bc_id[:4])
        self.ipfs = IPFS(self.bc_id)

    def authorizeClient(self, client):
        # hash is secret to the
        cid_hash = toHash(client.id_ + self.salt)
        self.clients.add(cid_hash)
        self.cli_pointers.append(client)

    def proofOfAuthority(self, clientId):
        if toHash(clientId + self.salt) not in self.clients:
            print("{0:5.5} MINE REJECTED: UNAUTH".format(clientId))
            return False
        print("{0:5.5} MINE APPROVED: POA".format(clientId))
        return True

    def newTransaction(self, trx):
        self.current_transactions.append(trx)
        return self.lastBlock.chain_idx + 1

    @property
    def lastBlock(self):
        return self.chain[-1]

    def newBlock(self, block):
        self.chain.append(block)
        self.current_transactions = []
        return block


In [13]:
classes_initial = [Node, Node, Node, Compounder, Node, OffChainNode, OffChainNode]
# At first, everyone is node | Assumed held global model
clients = setupClisF(train_inps, train_oups, test_inps, test_oups, rounds, [], classes_initial)
binds = [[6,3],[5,3]]
bindOCNodes(binds, clients)

In [14]:
blockchain = Blockchain()
# Assign blockchain
for cli in clients:
    if cli.__class__ != OffChainNode:
        cli.assignBlockchain(blockchain)

Blockchain created with ID: 53df


In [15]:

def printRepus(repus):
    for r in repus:
        print(r['hash'][:5], r['qual_mined'])

In [16]:
# for c in clients: train, send model to IPFS - get hash, send hash to blockchain as trx
# for m in miners: get trxs from blockchain backlog, 
def clientBatchRun(cli, epoch_, round_):
    # Train, send model to IPFS, get hash
    for rnd in range(round_):
        print('ROUND {} ==============================================================='.format(rnd))
        # Train off-chain first

        print('TRAINING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        # Train and submit ===============================================
        for ci in cli:
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                continue
            print(ci.hash[:5], 'in operation: TRAINING.')
            ###################### MOVE TO BC
            if ci.blockchain.proofOfAuthority(ci.id_) == False:
                print("Unauthorized")
                continue
            ci.cliTrainPack(epoch_, rnd)

        print('AGGREGATION>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        # Miners gather data ==============================================
        for ci in cli:
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                print('Class not Node')
                continue

            print(ci.hash[:5], 'in operation: MINE.')
            ci.aggregate(ci.blockchain)

        # REPU HERE FROM GLOBAL MODELS
        # CONSENSUS PART 2
        print('CONSENSUS>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        cli_new_inc = []
        for ci in cli:
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                print('Class not Node')
                continue
            new_increments = []
            increment = 0
            improving_clis = set()
            old_repus = truncateRepus(deepcopy(ci.blockchain.lastBlock.reputation_list))
            ev = ci.evaluate()[1]
            if len(old_repus) == 0:
                for t in blockchain.current_transactions:
                    if t.type == 'GLO' and t.sender == ci.hash:
                        c_hash, c_weight_glob_hash, c_repu_hash = t.load.split('|') # change repu hash to one's evaluation
                        c_repus = IPFStoPickleObj(IPFS(0), c_repu_hash, './temp/')
                        for r in c_repus:
                            print('CREPUS: -evaluate', r['qual_mined'], ev)
                            increment += float(r['qual_mined']) * ev
                            improving_clis.add(r['hash'])
            else:
                for t in blockchain.current_transactions:
                    if t.type == 'GLO':
                        c_hash, c_weight_glob_hash, c_repu_hash = t.load.split('|')
                        c_repus = IPFStoPickleObj(IPFS(0), c_repu_hash, './temp/')
                        print("CRPEUS: ",c_repus)

                        for i in old_repus:
                            print('RL =', i['hash'][:5], i['qual_mined'])
                        for i in c_repus:
                            print('RM =', i['hash'][:5], i['qual_mined'])

                        # compare with RL
                        tolerance = 0.95
                        for i in range(len(c_repus)): # WRONG LOGIC
                            for r in range(len(old_repus)):
                                if c_repus[i]['hash'] == old_repus[r]['hash'] and float(c_repus[i]['qual_mined']) > tolerance * float(old_repus[r]['qual_mined']):
                                    increment += (float(i['qual_mined']) - tolerance * float(old_repus[r]['qual_mined'])) * ci.evaluate()[1]
                                    c_repus[i] = old_repus[r]
                                    improving_clis.add(t.sender)

            new_increments.append({'miner_hash':ci.hash, 'increment':increment * log(ci.getVolume(),10), 'improving':improving_clis})
            print("NINCRE: ", end='')
            for ni in new_increments:
                print("NINCRE: ", ni['increment'])
            cli_new_inc.append(new_increments[0])

        print('WINNING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        #select max increment -> move to blockchain === CONSENSUS
        best = 0
        winning_miner = None
        for n in range(len(cli_new_inc)):
            #print(cli_new_inc[n]['miner_hash'], best, '>', cli_new_inc[n]['increment'])
            if cli_new_inc[n]['increment'] > best:
                best = cli_new_inc[n]['increment'] 
                winning_miner = cli_new_inc[n]['miner_hash']
                #print('winner changed to', cli_new_inc[n]['miner_hash'])
        try:
            print('Winner: ', winning_miner[:5])
            # exclude others' GLO
            for tx in blockchain.current_transactions:
                if tx.type == 'GLO' and tx.sender != winning_miner:
                    blockchain.current_transactions.remove(tx)
        except:
            print("No winners. Round skipped.")
            continue

        print('BLOCK PROPOSAL>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        # Client makes a block + merkleize, propose it to the blockchain
        proposed_block = None
        for ci in cli:
            if ci.hash == winning_miner:
                print(ci.hash[:5], 'in operation: BLOCK PUSH.')
                printRepus(ci.new_repus)
                proposed_block = Block(ci.blockchain, len(ci.blockchain.chain), ci.current_trxs, ci.hash, toHash(ci.blockchain.lastBlock), time(), ci.new_repus)
                ci.blockchain.newBlock(proposed_block) #IMPLEMENT
                print('Block created: ', toHash(ci.blockchain.lastBlock), ' with #trxs ', len(ci.blockchain.lastBlock.transactions), ' and miner ', ci.blockchain.lastBlock.proof[:5])
                print('Blockchain length now = ', len(ci.blockchain.chain))
                print("NREPU: ", ci.new_repus)
                print('TRXS:')
                for t in ci.blockchain.lastBlock.transactions:
                    print('tx', t.sender, t.type)

        print('GLOBAL IMPORT>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        # Clients import global model from winning miner
        for ci in cli:
            if ci.__class__ not in [Node, Compounder] or ci.committee == False:
                print('Class not Node')
                continue
            # import winning 
            for tx in ci.blockchain.lastBlock.transactions:
                if tx.type == 'GLO' and tx.sender == winning_miner:
                    weights = IPFStoPickleObj(ci.ipfs, tx.load.split('|')[1], './temp/')
                    ci.replaceWeights(weights) # Compounder needs imple here

        cli[0].blockchain.current_transactions = []

        print('QUALITY CONTROL>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')        
        # CHECK QUALITY
        for ci in cli:
            print('EVAL', ci.id_, ci.evaluate()[1])

        # if time for committee change:
            # do committee change
        print("FINAL REPUS: ")
        printRepus(blockchain.lastBlock.reputation_list)
        print('round {} done\n\n'.format(rnd))

In [17]:
clientBatchRun(clients, epochs, rounds)

ROUND 0 ===============================================================
TRAINING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
dfd5f in operation: TRAINING.
0     MINE APPROVED: POA
Cli# 0: 5.69 sec, Q: 0.7328; NDE:0 MOD-->CHIDX#  1
e2538 in operation: TRAINING.
1     MINE APPROVED: POA
Cli# 1: 3.69 sec, Q: 0.7295; NDE:1 MOD-->CHIDX#  1
58b2a in operation: TRAINING.
2     MINE APPROVED: POA
Cli# 2: 1.99 sec, Q: 0.6054; NDE:2 MOD-->CHIDX#  1
4cfc3 in operation: TRAINING.
3     MINE APPROVED: POA
Cli# 31da1a042dc910775ed8b487afbdafd929a7afdeaadc660cb963bd26: 0.62 sec, Q: 0.0874; Cli# b51d18b551043c1f145f22dbde6f8531faeaf68c54ed9dd79ce24d17: 0.64 sec, Q: 0.1792; 35/35 - 0s - loss: 0.0962 - accuracy: 0.1107
35/35 - 0s - loss: 0.0909 - accuracy: 0.1987
35/35 - 0s - loss: 0.1041 - accuracy: 0.1289
Cli# 3: 1.27 sec, Q: 0.3708; CPD:3 MOD-->CHIDX#  1
271f9 in operation: TRAINING.
4     MINE APPROVED: POA
Cli# 4: 1.06 sec, Q: 0.1683; NDE:4 MOD-->CHI